In [0]:
from scipy.cluster.vq import vq, kmeans2
from scipy import *
import datetime
import csv
import numpy as np
import scipy.sparse.linalg
import pandas as pd
import io

In [29]:
start_time = datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d %H:%M:%S')
print(start_time)

num_users = 1500001
num_items = 1358098
num_tests = 150000
num_clusters = 1000

userid_to_index = {}
users_matrix = []
itemid_to_index = {}
items_matrix = []
clust_to_index = {}
test_user_id = []
test_user_index = []
interaction_matrix = scipy.sparse.lil_matrix((num_users, num_items))

2018-06-27 13:39:56


In [30]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


Saving user_profile.csv to user_profile (1).csv
User uploaded file "user_profile.csv" with length 1615030 bytes


In [31]:
print("Reading user_profile.csv file ...")
users_rows = pd.read_csv(io.StringIO(uploaded["user_profile.csv"].decode('utf-8')), delimiter='\t')
# users_rows = csv.reader(open('../../data/training/users_indicators.csv'), delimiter='\t')
users_rows = users_rows.values
print(np.shape(users_rows))

Reading user_profile.csv file ...
(40000, 12)


In [32]:
print("Generating userid to index list ...")
user_counter = 0
for row in users_rows:
    userid_to_index[row[1]] = user_counter
    users_matrix.append(row)
    user_counter += 1
    if user_counter == num_users:
        break

Generating userid to index list ...


In [53]:
print("Reading items_profile.csv file ...")
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Reading items_profile.csv file ...


Saving item_profile.csv to item_profile (4).csv
User uploaded file "item_profile.csv" with length 21229339 bytes


In [54]:
items_rows = pd.read_csv(io.StringIO(uploaded["item_profile.csv"].decode('utf-8')), delimiter='\t')
items_rows = items_rows.values
print(np.shape(items_rows ))

(167956, 13)


In [0]:
# for row in items_rows:
#   titles = row[1].split(",")
#   tags = row[10].split(",")
#   print(len(titles))
#   print(len(tags))

In [55]:
print("Generating itemid to index list ...")
item_counter = 0
for row in items_rows:
    itemid_to_index[row[1]] = item_counter
    items_matrix.append(row)
    item_counter += 1
    if item_counter == num_items:
        break

Generating itemid to index list ...


In [0]:
index_to_items = {v: k for k, v in itemid_to_index.items()}

In [61]:
print("Generating items_matrix ...")
items_matrix_1 = np.array(items_matrix)
items_matrix_1[items_matrix_1 == ''] = '0'

print("Clustering items data ...")

centroids, item_clusts = kmeans2(np.delete(items_matrix_1, [0, 1, 2, 10, 5], 1).astype(float), num_clusters, minit='points')
# 10 should be splitted, 5 contains "ch" wtf

print("Appending clusters to last column of items_matrix_1 ...")
items_with_clusters = np.concatenate((items_matrix_1, item_clusts.reshape(item_clusts.shape + (1,))), 1)

Generating items_matrix ...
Clustering items data ...


/usr/local/lib/python3.6/dist-packages/scipy/cluster/vq.py:660: UserWarning: One of the clusters is empty. Re-run kmean with a different initialization.
  warnings.warn("One of the clusters is empty. "


Appending clusters to last column of items_matrix_1 ...


In [62]:
clust_to_items = scipy.sparse.lil_matrix((num_clusters, num_items))
for clust in range(0, num_clusters):
    items_in_clust = items_with_clusters[items_with_clusters[:, items_with_clusters.shape[1] - 1] == str(clust)][:, 1]
    for itemid in items_in_clust:
        clust_to_items[clust, itemid_to_index[itemid]] = 1
clust_to_items_1 = clust_to_items.tocsr()

item_time = datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d %H:%M:%S')
print(item_time)

2018-06-27 14:04:26


In [9]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving interactions.csv to interactions.csv
User uploaded file "interactions.csv" with length 15444858 bytes


In [10]:
print("Reading interactions.csv file ...")
interactions_reader = pd.read_csv(io.StringIO(uploaded["interactions.csv"].decode('utf-8')), delimiter='\t')

interactions_reader = interactions_reader.values
print(np.shape(interactions_reader))

Reading interactions.csv file ...
(546864, 4)


In [11]:
print("Generating a sparse users X items matrix for interaction ...")
num_lines_skipped = 0
num_matches = 0
for row in interactions_reader:
    if str(row[0]) in userid_to_index:
        num_matches += 1
        try:
            temp = float(row[2])
            if temp == 4.0:
                temp = -1.0
            interaction_matrix[userid_to_index[str(row[0])], itemid_to_index[str(row[1])]] = temp
        except:
            num_lines_skipped += 1

print(num_matches)
print(num_lines_skipped)
interaction_matrix_1 = interaction_matrix.tocsr() #Return a copy of this matrix in Compressed Sparse Row format

intr_time = datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d %H:%M:%S')
print(intr_time)

Generating a sparse users X items matrix for interaction ...
78
78
2018-06-27 12:49:22


In [63]:
print(np.shape(interaction_matrix))

(1500001, 1358098)


In [65]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving target_users.csv to target_users (1).csv
User uploaded file "target_users.csv" with length 76290 bytes


In [66]:
print("Reading target_users.csv file ...")

target_users_reader = pd.read_csv(io.StringIO(uploaded["target_users.csv"].decode('utf-8')), delimiter='\t')

target_users_reader = target_users_reader.values
print(np.shape(target_users_reader))

Reading target_users.csv file ...
(10000, 1)


In [67]:
for row in target_users_reader:
    if str(row[0]) in userid_to_index:
        test_index = userid_to_index[str(row[0])]
        test_user_id.append(str(row[0]))
        test_user_index.append(test_index)

print("Opening similarity output file ...")
fpResult = open('CF_ItemItemSimilarity.csv', 'w')

print("Generating new interaction vectors ...")
beta = interaction_matrix_1[test_user_index, :]

Opening similarity output file ...
Generating new interaction vectors ...


In [68]:
num_row = 0
for test_userid in test_user_id:
    num_items_written = 0
    test_index = userid_to_index[test_userid]
    beta = interaction_matrix_1[test_index, :]
    num_row += 1
    if num_row % 1000 == 0:
        print(num_row)

    alpha_z = []
    for clust in range (0, num_clusters):
        alpha = clust_to_items_1[clust, :]
        alpha_beta = beta.multiply(alpha)
        sum_row = alpha_beta.sum(1)[0, 0]
        num_nonzero = alpha_beta.nonzero()[1].__len__() + 0.001
        to_impute = sum_row/num_nonzero
        Z = np.repeat(to_impute, beta.shape[1])
        alpha_z = alpha.multiply(Z)
        idx = beta.nonzero()
        alpha_z[idx] = beta.data
    jobs_score = alpha_z.tolist()
    final_vector = np.argsort(jobs_score)[:, ::-1]

    tobeWritten = str(test_userid) + '\t'
    for item_index in final_vector[0]:
        item_id = index_to_items[int(item_index)]
        if items_with_clusters[int(item_index), 5] == str(1) and beta[0, item_index] != -1 and beta[0, item_index] != 3:
            tobeWritten += str(item_id)
            num_items_written += 1
            if num_items_written > 30:
                break
            tobeWritten += ','
    fpResult.write(tobeWritten + '\n')

end2_time = datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d %H:%M:%S')
print(end2_time)

TypeError: ignored